<a href="https://colab.research.google.com/github/QasimWani/Brain-Computer-Interface/blob/master/TD3_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gym==0.15.4
!pip install box2d-py
import gym
print(gym.__version__)

     |████████████████████████████████| 1.6MB 4.9MB/s 
     |████████████████████████████████| 1.0MB 31.8MB/s 
  Created wheel for gym: filename=gym-0.15.4-cp36-none-any.whl size=1648483 sha256=4558c3e6d890c9695b6e7e153e39889c44c21533e6b722e06654d27b5ed836af
  Stored in directory: /root/.cache/pip/wheels/e9/26/9b/8a1a6599a91077a938ac4348cc3d3ac84bfab0dbfddeb4c6e7
Successfully built gym
ERROR: tensorflow-probability 0.11.0 has requirement cloudpickle==1.3, but you'll have cloudpickle 1.2.2 which is incompatible.
  Found existing installation: pyglet 1.5.0
    Uninstalling pyglet-1.5.0:
      Successfully uninstalled pyglet-1.5.0
  Found existing installation: cloudpickle 1.3.0
    Uninstalling cloudpickle-1.3.0:
      Successfully uninstalled cloudpickle-1.3.0
  Found existing installation: gym 0.17.2
    Uninstalling gym-0.17.2:
      Successfully uninstalled gym-0.17.2
     |████████████████████████████████| 450kB 4.4MB/s 
0.15.4


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
cd drive/My\ Drive/TD3

/content/drive/My Drive/TD3


In [4]:
ls

models/


In [5]:
import numpy as np
from collections import deque, namedtuple
import torch
import random


#Implemented from: https://github.com/QasimWani/policy-value-methods/blob/master/DDPG/ddpg_agent.py#L150

#Set to cuda (gpu) instance if compute available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


BUFFER_SIZE = 50000 #max number of experiences in a buffer
MINI_BATCH = 256 #number of samples to collect from buffer

class ReplayBuffer():
    """
    Implementation of a fixed size replay buffer as used in DQN algorithms.
    The goal of a replay buffer is to unserialize relationships between sequential experiences, gaining a better temporal understanding.
    """
    def __init__(self, buffer_size=BUFFER_SIZE, batch_size=MINI_BATCH):
        """
        Initializes the buffer.
        @Param:
        1. action_size: env.action_space.shape[0]
        2. buffer_size: Maximum length of the buffer for extrapolating all experiences into trajectories. default - 1e6 (Source: DeepMind)
        3. batch_size: size of mini-batch to train on. default = 64.
        """
        self.replay_memory = deque(maxlen=buffer_size) #Experience replay memory object
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"]) #standard S,A,R,S',done
        
    def add(self, state, action, reward, next_state, done):
        """Adds an experience to existing memory"""
        trajectory = self.experience(state, action, reward, next_state, done)
        self.replay_memory.append(trajectory)
    
    def sample(self):
        """Randomly picks minibatches within the replay_buffer of size mini_batch"""
        experiences = random.sample(self.replay_memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).float().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)

        return states, actions, rewards, next_states, dones

    def __len__(self):#override default __len__ operator
        """Return the current size of internal memory."""
        return len(self.replay_memory)

In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

#Twin Delayed Deep Deterministic Policy Gradient (TD3) implementation for continuous action space control.
#Paper: https://arxiv.org/pdf/1802.09477.pdf
#Author: https://github.com/sfujim/TD3


class Actor(nn.Module):
    def __init__(self, state_size, action_size, max_action, fc1=256, fc2=256):
        """
        Initializes actor object.
        @Param:
        1. state_size: env.observation_space.shape[0].
        2. action_size: env.action_space.shape[0].
        3. max_action: abs(env.action_space.low), sets boundary/clip for policy approximation.
        4. fc1: number of hidden units for the first fully connected layer, fc1. Default = 256.
        5. fc2: number of hidden units for the second fully connected layer, fc1. Default = 256.
        """
        super(Actor, self).__init__()

        #Layer 1
        self.fc1 = nn.Linear(state_size, fc1)
        #Layer 2
        self.fc2 = nn.Linear(fc1, fc2)
        #Layer 3
        self.mu = nn.Linear(fc2, action_size)

        #Define boundary for action space.
        self.max_action = max_action
    
    def forward(self, state):
        """Peforms forward pass to map state--> pi(s)"""
        #Layer 1
        x = self.fc1(state)
        x = F.relu(x)
        #Layer 2
        x = self.fc2(x)
        x = F.relu(x)
        #Output layer
        mu = torch.tanh(self.mu(x))#set action b/w -1 and +1
        return self.max_action * mu


class Critic(nn.Module):
    def __init__(self, state_size, action_size, fc1=256, fc2=256):
        """
        Initializes Critic object, Q1 and Q2.
        Architecture different from DDPG. See paper for full details.
        @Param:
        1. state_size: env.observation_space.shape[0].
        2. action_size: env.action_space.shape[0].
        3. fc1: number of hidden units for the first fully connected layer, fc1. Default = 256.
        4. fc2: number of hidden units for the second fully connected layer, fc1. Default = 256.
        """
        super(Critic, self).__init__()

        #---------Q1 architecture---------
        
        #Layer 1
        self.l1 = nn.Linear(state_size + action_size, fc1)
        #Layer 2
        self.l2 = nn.Linear(fc1, fc2)
        #Output layer
        self.l3 = nn.Linear(fc2, 1)#Q-value

        #---------Q2 architecture---------

        #Layer 1
        self.l4 = nn.Linear(state_size + action_size, fc1)
        #Layer 2
        self.l5 = nn.Linear(fc1, fc2)
        #Output layer
        self.l6 = nn.Linear(fc2, 1)#Q-value
    
    def forward(self, state, action):
        """Perform forward pass by mapping (state, action) --> Q-value"""
        x = torch.cat([state, action], dim=1) #concatenate state and action such that x.shape = state.shape + action.shape

        #---------Q1 critic forward pass---------
        #Layer 1
        q1 = F.relu(self.l1(x))
        #Layer 2
        q1 = F.relu(self.l2(q1))
        #value prediction for Q1
        q1 = self.l3(q1)

        #---------Q2 critic forward pass---------
        #Layer 1
        q2 = F.relu(self.l4(x))
        #Layer 2
        q2 = F.relu(self.l5(q2))
        #value prediction for Q2
        q2 = self.l6(q2)

        return q1, q2

In [7]:
import random
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


#implementation from paper: https://arxiv.org/pdf/1802.09477.pdf
#source: https://github.com/sfujim/TD3/blob/master/TD3.py

#Set to cuda (gpu) instance if compute available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class Agent():
    """Agent that plays and learn from experience. Hyper-paramters chosen from paper."""
    def __init__(
            self, 
            state_size, 
            action_size, 
            max_action, 
            discount=0.99,
            tau=0.005,
            policy_noise=0.2,
            noise_clip=0.5,
            policy_freq=2
        ):
        """
        Initializes the Agent.
        @Param:
        1. state_size: env.observation_space.shape[0]
        2. action_size: env.action_size.shape[0]
        3. max_action: list of max values that the agent can take, i.e. abs(env.action_space.high)
        4. discount: return rate
        5. tau: soft target update
        6. policy_noise: noise reset level, DDPG uses Ornstein-Uhlenbeck process
        7. noise_clip: sets boundary for noise calculation to prevent from overestimation of Q-values
        8. policy_freq: number of timesteps to update the policy (actor) after
        """
        super(Agent, self).__init__()

        #Actor Network initialization
        self.actor = Actor(state_size, action_size, max_action).to(device)
        self.actor_target = copy.deepcopy(self.actor) #loads main model into target model
        self.actor_optimizer = torch.optim.Adam(self.actor.parameters(), lr=0.001)

        #Critic Network initialization
        self.critic = Critic(state_size, action_size).to(device)
        self.critic_target = copy.deepcopy(self.critic) #loads main model into target model
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(), lr=0.001)

        self.max_action = max_action
        self.discount = discount
        self.tau = tau
        self.policy_noise = policy_noise
        self.noise_clip = noise_clip
        self.policy_freq = policy_freq
        self.total_it = 0

    def select_action(self, state):
        """Selects an automatic epsilon-greedy action based on the policy"""
        state = torch.FloatTensor(state.reshape(1, -1)).to(device)
        return self.actor(state).cpu().data.numpy().flatten()
    
    def train(self, replay_buffer:ReplayBuffer):
        """Train the Agent"""

        self.total_it += 1

        # Sample replay buffer 
        state, action, reward, next_state, done = replay_buffer.sample()#sample 256 experiences

        with torch.no_grad():
            # Select action according to policy and add clipped noise
            noise = (
                torch.randn_like(action) * self.policy_noise
            ).clamp(-self.noise_clip, self.noise_clip)
            

            next_action = (
                self.actor_target(next_state) + noise #noise only set in training to prevent from overestimation
            ).clamp(-self.max_action, self.max_action)

            # Compute the target Q value
            target_Q1, target_Q2 = self.critic_target(next_state, next_action) #Q1, Q2
            target_Q = torch.min(target_Q1, target_Q2)
            target_Q = reward + done * self.discount * target_Q #TD-target

        # Get current Q estimates
        current_Q1, current_Q2 = self.critic(state, action) #Q1, Q2

        # Compute critic loss using MSE
        critic_loss = F.mse_loss(current_Q1, target_Q) + F.mse_loss(current_Q2, target_Q)

        # Optimize the critic
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        # Delayed policy updates (DDPG baseline = 1)
        if(self.total_it % self.policy_freq == 0):

            # Compute actor loss
            actor_loss = -self.critic(state, self.actor(state))[0].mean()
            
            # Optimize the actor 
            self.actor_optimizer.zero_grad()
            actor_loss.backward()
            self.actor_optimizer.step()

            # Soft update by updating the frozen target models
            for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
                target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)

            for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
                target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)


    def save(self, filename):
        """Saves the Actor Critic local and target models"""
        torch.save(self.critic.state_dict(), "models/checkpoint/"+ filename + "_critic")
        torch.save(self.critic_optimizer.state_dict(), "models/checkpoint/" + filename + "_critic_optimizer")

        torch.save(self.actor.state_dict(), "models/checkpoint/" + filename + "_actor")
        torch.save(self.actor_optimizer.state_dict(), "models/checkpoint/" + filename + "_actor_optimizer")


    def load(self, filename):
        """Loads the Actor Critic local and target models"""
        self.critic.load_state_dict(torch.load("models/checkpoint/" + filename + "_critic"))
        self.critic_optimizer.load_state_dict(torch.load("models/checkpoint/" + filename + "_critic_optimizer"))
        self.critic_target = copy.deepcopy(self.critic)

        self.actor.load_state_dict(torch.load("models/checkpoint/" + filename + "_actor"))
        self.actor_optimizer.load_state_dict(torch.load("models/checkpoint/" + filename + "_actor_optimizer"))
        self.actor_target = copy.deepcopy(self.actor)

In [ ]:
import numpy as np
import torch
import gym
import random
import matplotlib.pyplot as plt

env_id = "BipedalWalker-v2"
env = gym.make(env_id)


#set seeds
random_seed = 0
env.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
random.seed(random_seed)


#Set exploration noise for calculating action based on some noise factor
exploration_noise = 0.1

#Define observation and action space
state_space = env.observation_space.shape[0]
action_space = env.action_space.shape[0] 
max_action = float(env.action_space.high[0])

#Create Agent
policy = Agent(state_space, action_space, max_action)

# try:
#     policy.load("20")
# except:
#     raise IOError("Couldn't load policy")

#Create Replay Buffer
replay_buffer = ReplayBuffer()


#Train the model
max_episodes = 1000
max_timesteps = 2000

ep_reward = [] #get list of reward for range(max_episodes)

for episode in range(1, max_episodes+1):
    avg_reward = 0
    state = env.reset()
    for t in range(1, max_timesteps + 1):
        # select action and add exploration noise:
        action = policy.select_action(state) + np.random.normal(0, max_action * exploration_noise, size=action_space)
        action = action.clip(env.action_space.low, env.action_space.high)
            
        # take action in env:
        next_state, reward, done, _ = env.step(action)
        replay_buffer.add(state, action, reward, next_state, done)
        state = next_state
            
        avg_reward += reward

        #Renders an episode
        # env.render()
        if(len(replay_buffer) > 256):#make sure sample is less than overall population
            policy.train(replay_buffer) #training mode

        # if episode is done then update policy:
        if(done or t >=max_timesteps):
            print(f"Episode {episode} reward: {avg_reward} | Rolling average: {np.mean(ep_reward)}")
            print(f"Current time step: {t}")
            ep_reward.append(avg_reward)
            break 
    
    if(np.mean(ep_reward[-10:]) >= 300):
          policy.save("final")
          break

    if(episode % 100 == 0 and episode > 0):
        #Save policy and optimizer every 100 episodes
        policy.save(str("%02d" % (episode//100)))

env.close()

#Display Scores
fig = plt.figure()
plt.plot(np.arange(1, len(ep_reward) + 1), ep_reward)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Episode 1 reward: -125.4405564372999 | Rolling average: nan
Current time step: 331
Episode 2 reward: -111.61365066014513 | Rolling average: -125.4405564372999
Current time step: 60
Episode 3 reward: -107.66646206253465 | Rolling average: -118.52710354872252
Current time step: 78
Episode 4 reward: -126.34562858108363 | Rolling average: -114.90688971999323
Current time step: 88
Episode 5 reward: -136.08212325088542 | Rolling average: -117.76657443526584
Current time step: 375
Episode 6 reward: -210.11168341239187 | Rolling average: -121.42968419838977
Current time step: 943
Episode 7 reward: -114.30465781226322 | Rolling average: -136.21001740072344
Current time step: 125
Episode 8 reward: -110.85719937897036 | Rolling average: -133.0806803166577
Current time step: 37
Episode 9 reward: -108.04419271392985 | Rolling average: -130.30274519944678
Current time step: 74
Episode 10 reward: -105.84954205352483 | Rolling average: -127.829572701056
Current time step: 52
Episode 11 reward: -106.31